In [1]:
import requests
import pandas as pd
from tqdm.auto import tqdm
import time

In [6]:
def query_nyt(query, your_key):
    
    for date_begin,date_end in list(zip(range(19950101,20220101,10000),range(19960101,20230101,10000))):
        filt = f'begin_date={date_begin}&end_date={date_end}'
        query_url_articles = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={query}&{filt}&api-key={your_key}'
        articles_request = requests.get(query_url_articles)
        results = articles_request.json()
        if results.get('fault') is None:
            n_results = results['response']['meta']['hits']
            articles_list_df = []
            for i in tqdm(range(0,int(n_results/10))):
                query_url_articles_page = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={query}&{filt}&page={i}&api-key={your_key}'
                articles_request = requests.get(query_url_articles_page)
                results = articles_request.json()
                if results.get('fault') is None:
                    df = pd.json_normalize(results['response']['docs'])
                    df['query_consulta']=query
                    articles_list_df.append(df)
                    time.sleep(6)
                else:
                    pd.concat(articles_list_df).to_csv(f'{query}_{date_begin}_{date_end}.csv',index=False)
                    return date_begin,i
            pd.concat(articles_list_df).to_csv(f'{query}_{date_begin}_{date_end}.csv',index=False)
    return date_begin,i